<a href="https://colab.research.google.com/github/yasminemasmoudi/P2M/blob/master/Fix_Missing_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import requests
drive_path = '/content/drive/My Drive/p2mfiles/'
from bs4 import BeautifulSoup

df_books = pd.read_csv(drive_path + 'to_clean.csv')
nopic_ids = df_books[df_books.image_url.str.contains('nophoto')].book_id.values

headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})
URL = 'https://www.goodreads.com/book'

print(f'Total missing images : {len(nopic_ids)}')
no_images = set([389627,42155,4502877,25735012,19469, 3698,207781, 53580, 105703,63432,313620, 78674, 44360 ])
ex = 0

for i, bookid in enumerate(nopic_ids):
    if bookid in no_images:
      continue
    try:
        page = requests.get(URL + str(bookid), headers = headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        new_image_url = soup.find("div", {"class": "editionCover"}).img['src']
        df_books.loc[df_books.book_id == bookid,'image_url'] = new_image_url
    except Exception as e:
        ex += 1
        print(f'No image found for Bookid : {bookid} due to {e}')
        if(ex > 5):
          print('Terminating ..')
          break
    if i and i % 100 == 0:
        print(f'Processed {i} Books')

new_nopic_ids = df_books[df_books.image_url.str.contains('nophoto')].book_id.values
print(f'Missing images now: {len(new_nopic_ids)}')
print(f'Updated {abs(len(new_nopic_ids) - len(nopic_ids))} images')
df_books.to_csv(drive_path + 'to_clean.csv', index = False)

Total missing images : 13
Missing images now: 13
Updated 0 images


In [9]:
cols = ['original_title', 'authors', 'average_rating', 'image_url', 'book_id']

In [10]:
cols = [c for c in df_books.columns if not 'Unnamed' in c]
df = df_books[cols].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9274 entries, 0 to 9273
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   r_index                    9274 non-null   int64  
 1   book_id                    9274 non-null   int64  
 2   best_book_id               9274 non-null   int64  
 3   work_id                    9274 non-null   int64  
 4   books_count                9274 non-null   int64  
 5   isbn                       8728 non-null   object 
 6   isbn13                     8828 non-null   float64
 7   authors                    9274 non-null   object 
 8   original_publication_year  9268 non-null   float64
 9   original_title             9274 non-null   object 
 10  title                      9274 non-null   object 
 11  language_code              8281 non-null   object 
 12  average_rating             9274 non-null   float64
 13  ratings_count              9274 non-null   int64

In [11]:
df.to_csv('full_books.csv', index = False)

In [12]:
df.to_csv(drive_path + 'full_books.csv', index = False)

In [13]:
cols = ['r_index','book_id','authors','original_title','image_url','average_rating','ratings_count']
df = df[cols]
df.to_csv('books.csv', index = False)